In [1]:
from lark import Lark, Transformer, v_args


# try:
#     input = raw_input   # For Python2 compatibility
# except NameError:
#     pass


calc_grammar = """
    ?start : expr
        | NAME "=" expr     -> assign_var

    ?expr : expr "+" term    ->add
        |  term

    ?term : term "*" factor  ->mul
        |  factor

    ?factor :  "(" expr ")"
        | NAME              -> var
        | NUMBER             -> number
 
    %import common.CNAME -> NAME
    %import common.NUMBER
    %import common.WS_INLINE
    %ignore WS_INLINE
"""


@v_args(inline=True)    # Affects the signatures of the methods
class CalculateTree(Transformer):
    from operator import add, mul
    number = float

    def __init__(self):
        self.vars = {}

    def assign_var(self, name, value):
        self.vars[name] = value
        return value

    def var(self, name):
        return self.vars[name]


calc_parser = Lark(calc_grammar, parser='lalr', transformer=CalculateTree())
tree = Lark(calc_grammar)
calc = calc_parser.parse


def main():
    i = 0
    while True:
        try:
            s = input('> ')
        except EOFError:
            break
        
        print(calc(s))
        if(i!=0 and i%3==0):
            print(tree.parse(s))
            break
            
        i = i+1


def test():
    print(calc("a = 1+2"))
    print(calc("1+a*-3"))


if __name__ == '__main__':
     #test()
    main()

> a=1
1.0
> b=2
2.0
> c=3
3.0
> ((a+b)*a+c)*a+b*c
12.0
Tree(add, [Tree(mul, [Tree(add, [Tree(mul, [Tree(add, [Tree(var, [Token(NAME, 'a')]), Tree(var, [Token(NAME, 'b')])]), Tree(var, [Token(NAME, 'a')])]), Tree(var, [Token(NAME, 'c')])]), Tree(var, [Token(NAME, 'a')])]), Tree(mul, [Tree(var, [Token(NAME, 'b')]), Tree(var, [Token(NAME, 'c')])])])
